## GSE180286 dataset

In [1]:
import os
import tarfile
import gzip
import pandas as pd
import scanpy as sc
download_url = "https://ftp.ncbi.nlm.nih.gov/geo/series/GSE180nnn/GSE180286/suppl/GSE180286_RAW.tar"
raw_dir = os.path.expanduser("~/Desktop/Capstone/Raw Data")
tar_path = os.path.join(raw_dir, "GSE180286_RAW.tar")

# Step 1: Download the file
import urllib.request
if not os.path.exists(tar_path):
    print("Downloading...")
    urllib.request.urlretrieve(download_url, tar_path)
    print("Download completed.")
else:
    print("File already exists.")

extracted_dir = os.path.join(raw_dir, "GSE180286_EXTRACTED")
os.makedirs(extracted_dir, exist_ok=True)

with tarfile.open(tar_path, "r") as tar:
    tar.extractall(path=extracted_dir)
    print("Extraction complete.")
files = sorted([f for f in os.listdir(extracted_dir) if f.endswith(".txt.gz")])
print("Available files:", files[:5])  # Show a few

# Choose one file
expr_file = os.path.join(extracted_dir, files[0])
print(f"Loading file: {expr_file}")
with gzip.open(expr_file, 'rt') as f:
    df = pd.read_csv(f, sep="\t", index_col=0)

adata3 = sc.AnnData(df.T)
adata3.var_names_make_unique()

File already exists.
Extraction complete.
Available files: ['GSM5457199_A2019-1.expression_matrix.txt.gz', 'GSM5457200_A2019-2.expression_matrix.txt.gz', 'GSM5457201_A2019-3.expression_matrix.txt.gz', 'GSM5457202_B2019-1.expression_matrix.txt.gz', 'GSM5457203_B2019-2.expression_matrix.txt.gz']
Loading file: /Users/adi/Desktop/Capstone/Raw Data/GSE180286_EXTRACTED/GSM5457199_A2019-1.expression_matrix.txt.gz


In [2]:
adatas = []

for fname in files:
    path = os.path.join(extracted_dir, fname)
    print(f"Loading: {path}")
    
    with gzip.open(path, 'rt') as f:
        df = pd.read_csv(f, sep="\t", index_col=0)
    
    ad = sc.AnnData(df.T)
    ad.var_names_make_unique()
    
    # Add metadata: sample ID (e.g., GSM5457199)
    sample_id = fname.split("_")[0]
    ad.obs["sample"] = sample_id
    
    adatas.append(ad)

# Concatenate all into one AnnData object
adata3 = adatas[0].concatenate(
    adatas[1:], 
    batch_key="file", 
    batch_categories=[f.split("_")[0] for f in files]
)
print("✅ Combined adata3 shape:", adata3.shape)

Loading: /Users/adi/Desktop/Capstone/Raw Data/GSE180286_EXTRACTED/GSM5457199_A2019-1.expression_matrix.txt.gz
Loading: /Users/adi/Desktop/Capstone/Raw Data/GSE180286_EXTRACTED/GSM5457200_A2019-2.expression_matrix.txt.gz
Loading: /Users/adi/Desktop/Capstone/Raw Data/GSE180286_EXTRACTED/GSM5457201_A2019-3.expression_matrix.txt.gz
Loading: /Users/adi/Desktop/Capstone/Raw Data/GSE180286_EXTRACTED/GSM5457202_B2019-1.expression_matrix.txt.gz
Loading: /Users/adi/Desktop/Capstone/Raw Data/GSE180286_EXTRACTED/GSM5457203_B2019-2.expression_matrix.txt.gz
Loading: /Users/adi/Desktop/Capstone/Raw Data/GSE180286_EXTRACTED/GSM5457204_B2019-3.expression_matrix.txt.gz
Loading: /Users/adi/Desktop/Capstone/Raw Data/GSE180286_EXTRACTED/GSM5457205_C2020-1.expression_matrix.txt.gz
Loading: /Users/adi/Desktop/Capstone/Raw Data/GSE180286_EXTRACTED/GSM5457206_C2020-2.expression_matrix.txt.gz
Loading: /Users/adi/Desktop/Capstone/Raw Data/GSE180286_EXTRACTED/GSM5457207_C2020-3.expression_matrix.txt.gz
Loading: /

/var/folders/kx/1z2nljtd7zqgfpqmrmskdmqc0000gn/T/ipykernel_942/451958691.py:20: FutureWarning: Use anndata.concat instead of AnnData.concatenate, AnnData.concatenate is deprecated and will be removed in the future. See the tutorial for concat at: https://anndata.readthedocs.io/en/latest/concatenation.html
  adata3 = adatas[0].concatenate(


✅ Combined adata3 shape: (117481, 17872)


In [4]:
print(adata3.obs.head())
adata3.obs['file'].value_counts()
adata3.obs['sample'].value_counts()
# adata3.obs['percent.mito']

                             sample        file
GGATAAGGGTCA-GSM5457199  GSM5457199  GSM5457199
CCGTGCGTACTG-GSM5457199  GSM5457199  GSM5457199
AGGTAACCTACG-GSM5457199  GSM5457199  GSM5457199
CTGTATAACCTA-GSM5457199  GSM5457199  GSM5457199
AAACAGGTTTGA-GSM5457199  GSM5457199  GSM5457199


sample
GSM5457201    16661
GSM5457202    11467
GSM5457203    11356
GSM5457204    11323
GSM5457200     8607
GSM5457207     8517
GSM5457210     8228
GSM5457211     7521
GSM5457213     6398
GSM5457212     5673
GSM5457209     5298
GSM5457206     4940
GSM5457205     4161
GSM5457208     4064
GSM5457199     3267
Name: count, dtype: int64

In [ ]:
## Don't run this code block
import os
import tarfile
import gzip
import pandas as pd
import scanpy as sc

# Paths
raw_dir = os.path.expanduser("~/Desktop/Capstone/Raw Data")
extracted_dir = os.path.join(raw_dir, "GSE180286_EXTRACTED")
files = sorted([f for f in os.listdir(extracted_dir) if f.endswith(".txt.gz")])

# Load all files into AnnData objects
adatas = []
for fname in files:
    path = os.path.join(extracted_dir, fname)
    print(f"🔹 Loading: {fname}")
    
    with gzip.open(path, 'rt') as f:
        df = pd.read_csv(f, sep="\t", index_col=0)
    
    # Genes = rows, Cells = columns → Transpose!
    ad = sc.AnnData(df.T)
    ad.var_names_make_unique()

    sample_id = fname.split("_")[0]
    ad.obs["sample"] = sample_id
    adatas.append(ad)

# Concatenate all samples
adata3 = sc.concat(adatas, label="file", keys=[f.split("_")[0] for f in files])
print("✅ Final shape:", adata3.shape)

🔹 Loading: GSM5457199_A2019-1.expression_matrix.txt.gz
🔹 Loading: GSM5457200_A2019-2.expression_matrix.txt.gz
🔹 Loading: GSM5457201_A2019-3.expression_matrix.txt.gz
🔹 Loading: GSM5457202_B2019-1.expression_matrix.txt.gz
🔹 Loading: GSM5457203_B2019-2.expression_matrix.txt.gz
🔹 Loading: GSM5457204_B2019-3.expression_matrix.txt.gz
🔹 Loading: GSM5457205_C2020-1.expression_matrix.txt.gz
🔹 Loading: GSM5457206_C2020-2.expression_matrix.txt.gz
🔹 Loading: GSM5457207_C2020-3.expression_matrix.txt.gz
🔹 Loading: GSM5457208_D2020-1.expression_matrix.txt.gz
🔹 Loading: GSM5457209_D2020-2.expression_matrix.txt.gz
🔹 Loading: GSM5457210_D2020-3.expression_matrix.txt.gz
🔹 Loading: GSM5457211_E2020-1.expression_matrix.txt.gz
🔹 Loading: GSM5457212_E2020-2.expression_matrix.txt.gz
🔹 Loading: GSM5457213_E2020-3.expression_matrix.txt.gz
✅ Final shape: (117481, 17872)


/Users/adi/Library/Python/3.9/lib/python/site-packages/anndata/_core/anndata.py:1754: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


In [1]:
import scanpy as sc
import pandas as pd
import scanpy as sc

# Load the feature file
features_path = "/Users/adi/Desktop/Capstone/Notebook recent/GSE180286_obs_features.csv"
df_features = pd.read_csv(features_path, index_col=0)

# Check alignment
common_obs = df_features.index.intersection(adata3.obs_names)

# Filter only common cells
df_features_aligned = df_features.loc[common_obs].copy()

# Drop overlapping columns from the features dataframe
df_features_aligned_clean = df_features_aligned.drop(columns=['sample', 'file'], errors='ignore')

# Now perform the join
adata3.obs = adata3.obs.join(df_features_aligned_clean)

print("✅ Successfully joined. Columns in adata3.obs:")
print(adata3.obs.columns.tolist())


NameError: name 'adata3' is not defined

In [5]:
pip install gtfparse


Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [5]:
# Common mitochondrial Ensembl gene ID prefixes: ENSG00000198888 → MT-ND1, etc.
mt_gene_names = [
    "MT-ND1", "MT-ND2", "MT-ND3", "MT-ND4", "MT-ND4L", "MT-ND5", "MT-ND6",
    "MT-CO1", "MT-CO2", "MT-CO3",
    "MT-ATP6", "MT-ATP8",
    "MT-CYB", "MT-RNR1", "MT-RNR2", "MT-TP", "MT-TL1", "MT-TL2"
]

# Check if these are present
mt_present = [gene for gene in mt_gene_names if gene in adata3.var_names]
print("✅ MT genes found:", mt_present)



✅ MT genes found: ['MT-ND1', 'MT-ND2', 'MT-ND3', 'MT-ND4', 'MT-ND4L', 'MT-ND5', 'MT-ND6', 'MT-CO1', 'MT-CO2', 'MT-CO3', 'MT-ATP6', 'MT-ATP8', 'MT-CYB', 'MT-RNR1', 'MT-RNR2', 'MT-TP', 'MT-TL1', 'MT-TL2']


In [6]:
import os
import urllib.request
from gtfparse import read_gtf

# Step 1: Download GTF if needed
gtf_url = "https://ftp.ebi.ac.uk/pub/databases/gencode/Gencode_human/release_44/gencode.v44.annotation.gtf.gz"
gtf_file = "gencode.v44.annotation.gtf.gz"

if not os.path.exists(gtf_file):
    print("📥 Downloading GTF file...")
    urllib.request.urlretrieve(gtf_url, gtf_file)
else:
    print("✅ GTF file already exists.")

# Step 2: Read GTF
gtf_df = read_gtf(gtf_file).to_pandas()

# Step 3: Extract gene_id and gene_name
genes = gtf_df[gtf_df["feature"] == "gene"][["gene_id", "gene_name"]].drop_duplicates()

# Step 4: Strip version from Ensembl IDs
genes["gene_id_clean"] = genes["gene_id"].str.replace(r"\..*", "", regex=True)

# Step 5: Create mapping
ens_to_symbol = dict(zip(genes["gene_id_clean"], genes["gene_name"]))

# Step 6: Map gene symbols in adata_all.var
adata3.var["ensembl_id"] = adata3.var_names.str.replace(r"\..*", "", regex=True)
adata3.var["gene_name"] = adata3.var["ensembl_id"].map(ens_to_symbol)

# Step 7: Fill missing symbols with Ensembl ID, then set .var_names
# Step 7: Fill NaNs and ensure all entries are strings
adata3.var["gene_name_clean"] = (
    adata3.var["gene_name"].fillna(adata3.var["ensembl_id"])
).astype(str)

# Set .var_names safely
adata3.var_names = adata3.var["gene_name_clean"]
adata3.var_names_make_unique()


# ✅ Check first few gene names
print(adata3.var_names[:10].tolist())

✅ GTF file already exists.


INFO:root:Extracted GTF attributes: ['gene_id', 'gene_type', 'gene_name', 'level', 'tag', 'transcript_id', 'transcript_type', 'transcript_name', 'transcript_support_level', 'havana_transcript', 'exon_number', 'exon_id', 'hgnc_id', 'havana_gene', 'ont', 'protein_id', 'ccdsid', 'artif_dupl']


['UQCR10', 'SPP1', 'HLA-DPB1', 'MT-RNR2', 'RPS29', 'B2M', 'RAB14', 'HMGN2', 'NDUFAF6', 'AKR1A1']


In [7]:
# Identify mitochondrial genes by prefix
adata3.var['mt'] = adata3.var_names.str.upper().str.startswith('MT-')

# Recalculate QC metrics with updated mitochondrial flags
import scanpy as sc
sc.pp.calculate_qc_metrics(adata3, qc_vars=['mt'], inplace=True)

# Assign percent.mito (to match GSE176078 column naming)
adata3.obs['percent.mito'] = adata3.obs['pct_counts_mt']



In [8]:
# Confirm if any MT- genes exist and their total expression
mt_genes = adata3.var_names[adata3.var_names.str.upper().str.startswith("MT-")]
print("🔍 MT-genes found:", mt_genes.tolist())

# Check summed expression of mitochondrial genes
mt_sum = adata3[:, mt_genes].X.sum()
total_sum = adata3.X.sum()
print(f"MT-total expression: {mt_sum}")
print(f"Total expression: {total_sum}")

🔍 MT-genes found: ['MT-RNR2', 'MT-CYB', 'MT-RNR1', 'MT-ND2', 'MT-CO1', 'MT-ND3', 'MT-CO3', 'MT-ATP6', 'MT-ND4', 'MT-ND4L', 'MT-ND5', 'MT-CO2', 'MT-ATP8', 'MT-TE', 'MT-ND1', 'MT-TR', 'MT-TW', 'MT-TP', 'MT-TT', 'MT-TV', 'MT-TL1', 'MT-TH', 'MT-ND6', 'MT-TM', 'MT-TF', 'MT-TS2', 'MT-TL2', 'MT-TI', 'MT-TD', 'MT-TG', 'MT-TQ', 'MT-TC', 'MT-TY', 'MT-TS1', 'MT-TA']
MT-total expression: 21227320
Total expression: 202277972


In [9]:
adata3.obs['percent.mito']


GGATAAGGGTCA-GSM5457199    20.959424
CCGTGCGTACTG-GSM5457199    18.918919
AGGTAACCTACG-GSM5457199    18.624554
CTGTATAACCTA-GSM5457199    17.816680
AAACAGGTTTGA-GSM5457199    20.350034
                             ...    
CTCCAACCAATG-GSM5457213    12.745098
CTCCTGAAGCAC-GSM5457213     5.263158
GTACCAGCGGCA-GSM5457213     6.737589
TACCTCCTAAAG-GSM5457213    30.769231
TGGTTTGTAGGG-GSM5457213    18.411552
Name: percent.mito, Length: 117481, dtype: float64

In [9]:
del adata3

# Then force garbage collection
import gc
gc.collect()

50

In [10]:
import os
import urllib.request
import pandas as pd
from gtfparse import read_gtf
import os
import urllib.request
import pandas as pd
from gtfparse import read_gtf

# ✅ Step 1: Download GTF file if not present
gtf_url = "https://ftp.ebi.ac.uk/pub/databases/gencode/Gencode_human/release_44/gencode.v44.annotation.gtf.gz"
gtf_file = "gencode.v44.annotation.gtf.gz"

if not os.path.exists(gtf_file):
    print("📥 Downloading GTF file...")
    urllib.request.urlretrieve(gtf_url, gtf_file)
else:
    print("✅ GTF file already exists.")

# ✅ Step 2: Parse GTF and extract gene features
gtf_df = read_gtf(gtf_file).to_pandas()
genes = gtf_df[gtf_df["feature"] == "gene"]
gene_pos = genes[["gene_name", "seqname", "start", "end"]].drop_duplicates()
gene_pos.columns = ["gene", "chromosome", "start", "end"]

# ✅ Step 3: Filter gene positions to only those in adata3
adata3.var["gene"] = adata3.var_names
gene_pos_filtered = gene_pos[gene_pos["gene"].isin(adata3.var["gene"])].drop_duplicates(subset="gene")
gene_pos_filtered = gene_pos_filtered.set_index("gene")

# ✅ Step 4: Remove any existing coordinate columns that would conflict
adata3.var = adata3.var.drop(columns=["chromosome", "start", "end"], errors="ignore")

# ✅ Step 5: Join and reindex
adata3.var = adata3.var.join(gene_pos_filtered, on="gene")
adata3.var = adata3.var.reindex(adata3.var_names)

print("✅ Gene coordinate columns added to adata3.var:")
print(adata3.var[["chromosome", "start", "end"]].head())



✅ GTF file already exists.


INFO:root:Extracted GTF attributes: ['gene_id', 'gene_type', 'gene_name', 'level', 'tag', 'transcript_id', 'transcript_type', 'transcript_name', 'transcript_support_level', 'havana_transcript', 'exon_number', 'exon_id', 'hgnc_id', 'havana_gene', 'ont', 'protein_id', 'ccdsid', 'artif_dupl']


✅ Gene coordinate columns added to adata3.var:
                chromosome       start         end
gene_name_clean                                   
UQCR10               chr22  29767369.0  29770413.0
SPP1                  chr4  87975667.0  87983532.0
HLA-DPB1              chr6  33075990.0  33089696.0
MT-RNR2               chrM      1671.0      3229.0
RPS29                chr14  49570984.0  49599164.0


In [11]:
adata3.obs["sample"].value_counts()
adata3.obs["cnv_reference"] = "tumor"
adata3.obs.loc[adata3.obs["sample"] == "GSM5457199", "cnv_reference"] = "normal"
print(adata3.obs["cnv_reference"].value_counts())
print(adata3.obs["percent.mito"].value_counts())


cnv_reference
tumor     114214
normal      3267
Name: count, dtype: int64
percent.mito
7.692308     151
8.333333     145
9.090909     135
7.142857     131
6.250000     126
            ... 
4.496403       1
6.055209       1
10.753676      1
6.893382       1
18.411552      1
Name: count, Length: 65110, dtype: int64


In [12]:
# Clean chromosome names
adata3.var["chromosome"] = (
    adata3.var["chromosome"].astype(str)
    .str.replace("chr", "", regex=False)
    .str.upper()
)

# Show sample values
print("✅ Sample chromosome values:", adata3.var["chromosome"].unique().tolist()[:10])

# Keep canonical chromosomes only
valid_chroms = [str(i) for i in range(1, 23)] + ["X", "Y"]
mask = adata3.var["chromosome"].isin(valid_chroms)
print(f"✅ Genes with valid chromosome info: {mask.sum()} / {adata3.shape[1]}")

# Subset the full object to valid chromosomes only
adata3 = adata3[:, mask].copy()






✅ Sample chromosome values: ['22', '4', '6', 'M', '14', '15', '9', '1', '8', 'NAN']
✅ Genes with valid chromosome info: 14773 / 17872


In [13]:

import numpy as np
import pandas as pd
from collections import defaultdict

# Clean chromosome info
adata3.var["chromosome"] = (
    adata3.var["chromosome"].astype(str)
    .str.replace("chr", "")
    .str.upper()
)

# Keep only genes with chromosome info
valid_mask = adata3.var["chromosome"].notna()
valid_genes = adata3.var[valid_mask]

# Extract matrix and chromosome info
X_cnv = adata3.X[:, valid_mask]  # ✅ Use .X, not "X_cnv"
chroms = valid_genes["chromosome"].values

# Group gene indices by chromosome
chrom_to_idx = defaultdict(list)
for idx, chrom in enumerate(chroms):
    chrom_to_idx[chrom].append(idx)

# Create dataframe for CNV signal
cnv_chr_df = pd.DataFrame(index=adata3.obs_names)

# Compute average signal per chromosome
for chrom, indices in chrom_to_idx.items():
    X_chr = X_cnv[:, indices]
    mean_signal = np.asarray(X_chr.mean(axis=1)).ravel()
    cnv_chr_df[chrom] = mean_signal

print("✅ Chromosome-wise CNV matrix shape:", cnv_chr_df.shape)




✅ Chromosome-wise CNV matrix shape: (117481, 24)


In [14]:
# Step 4: Compute CNV score as mean absolute deviation across chromosomes
cnv_score = cnv_chr_df.abs().mean(axis=1)

# Store in AnnData object
adata3.obs["cnv_score"] = cnv_score

# ✅ Check result
print("✅ Sample CNV scores:")
print(adata3.obs[["cnv_score", "cnv_reference"]].head())


✅ Sample CNV scores:
                         cnv_score cnv_reference
GGATAAGGGTCA-GSM5457199   3.085859        normal
CCGTGCGTACTG-GSM5457199   2.003530        normal
AGGTAACCTACG-GSM5457199   1.520273        normal
CTGTATAACCTA-GSM5457199   1.310010        normal
AAACAGGTTTGA-GSM5457199   1.156577        normal


In [15]:
adata3.obs['nCount_RNA'] = adata3.X.sum(axis=1).A1 if hasattr(adata3.X, 'A1') else adata3.X.sum(axis=1)
mito_genes = adata3.var_names.str.startswith('MT-')
adata3.obs['percent.mito'] = (adata3[:, mito_genes].X.sum(axis=1).A1 if hasattr(adata3.X, 'A1') 
                              else adata3[:, mito_genes].X.sum(axis=1)) / adata3.obs['nCount_RNA'] * 100

In [17]:
ribo_genes = adata3.var_names.str.upper().str.startswith(('RPS', 'RPL'))
adata3.obs['pct_counts_ribo'] = (adata3[:, ribo_genes].X.sum(axis=1).A1 if hasattr(adata3.X, 'A1') 
                                 else adata3[:, ribo_genes].X.sum(axis=1)) / adata3.obs['nCount_RNA'] * 100


In [18]:
adata3.obs.head(2)

,sample,file,n_genes_by_counts,log1p_n_genes_by_counts,total_counts,log1p_total_counts,pct_counts_in_top_50_genes,pct_counts_in_top_100_genes,pct_counts_in_top_200_genes,pct_counts_in_top_500_genes,total_counts_mt,log1p_total_counts_mt,pct_counts_mt,percent.mito,cnv_reference,cnv_score,nCount_RNA,pct_counts_ribo
GGATAAGGGTCA-GSM5457199,GSM5457199,GSM5457199,6931,8.843904,61662,11.029439,35.125361,43.456262,52.221790,64.303785,12924,9.466919,20.959424,0.0,normal,3.085859,47478,18.918236
CCGTGCGTACTG-GSM5457199,GSM5457199,GSM5457199,5618,8.633909,39183,10.576024,33.233800,41.745145,51.570834,65.099661,7413,8.911125,18.918919,0.0,normal,2.003530,30785,18.080234


In [19]:
# Seurat v3 cell cycle gene sets
s_genes = [
    'MCM5', 'PCNA', 'TYMS', 'FEN1', 'MCM2', 'MCM4', 'RRM1', 'UNG', 'GINS2',
    'MCM6', 'CDCA7', 'DTL', 'PRIM1', 'UHRF1', 'MCM10', 'HELLS', 'RFC2', 'RPA2',
    'NASP', 'RAD51AP1', 'GMNN', 'WDR76', 'SLBP', 'CCNE2', 'UBR7', 'POLD3',
    'MSH2', 'ATAD2', 'RAD51', 'RRM2', 'CDC45', 'CDC6', 'EXO1', 'TIPIN', 'DSCC1',
    'BLM', 'CASP8AP2', 'USP1', 'CLSPN', 'POLA1', 'CHAF1B', 'BRIP1', 'E2F8'
]

g2m_genes = [
    'HMGB2', 'CDK1', 'NUSAP1', 'UBE2C', 'BIRC5', 'TPX2', 'TOP2A', 'NDC80',
    'CKS2', 'NUF2', 'CKS1B', 'MKI67', 'TMPO', 'CENPF', 'TACC3', 'FAM64A',
    'SMC4', 'CCNB2', 'CKAP2L', 'CKAP2', 'AURKB', 'BUB1', 'KIF11', 'ANP32E',
    'TUBB4B', 'GTSE1', 'KIF20B', 'HJURP', 'CDC20', 'TTK', 'CDC25C', 'KIF2C',
    'RANGAP1', 'NCAPD2', 'DLGAP5', 'CDCA3', 'HN1', 'CDC45', 'CDCA8', 'ECT2',
    'KIF23', 'HMMR', 'AURKA', 'PSRC1', 'ANLN', 'LBR', 'CKAP5', 'CENPE',
    'CTCF', 'NEK2', 'G2E3', 'GAS2L3', 'CBX5', 'CENPA'
]

# ✅ Filter genes present in your dataset
s_genes_present = [g for g in s_genes if g in adata3.var_names]
g2m_genes_present = [g for g in g2m_genes if g in adata3.var_names]

print(f"S phase genes found: {len(s_genes_present)}")
print(f"G2M phase genes found: {len(g2m_genes_present)}")

# ✅ Score cell cycle
sc.tl.score_genes_cell_cycle(adata3, s_genes=s_genes_present, g2m_genes=g2m_genes_present)

# ✅ Confirm it worked
print(adata3.obs.head())

S phase genes found: 43
G2M phase genes found: 52
                             sample        file  n_genes_by_counts  \
GGATAAGGGTCA-GSM5457199  GSM5457199  GSM5457199               6931   
CCGTGCGTACTG-GSM5457199  GSM5457199  GSM5457199               5618   
AGGTAACCTACG-GSM5457199  GSM5457199  GSM5457199               5122   
CTGTATAACCTA-GSM5457199  GSM5457199  GSM5457199               4799   
AAACAGGTTTGA-GSM5457199  GSM5457199  GSM5457199               4634   

                         log1p_n_genes_by_counts  total_counts  \
GGATAAGGGTCA-GSM5457199                 8.843904         61662   
CCGTGCGTACTG-GSM5457199                 8.633909         39183   
AGGTAACCTACG-GSM5457199                 8.541495         29445   
CTGTATAACCTA-GSM5457199                 8.476371         25420   
AAACAGGTTTGA-GSM5457199                 8.441391         23312   

                         log1p_total_counts  pct_counts_in_top_50_genes  \
GGATAAGGGTCA-GSM5457199           11.029439              

In [20]:
# Define OxPhos genes
oxphos_genes = [
    "ATP5F1A", "ATP5F1B", "ATP5MC1", "ATP5MC2", "ATP5ME", "ATP5MG",
    "COX4I1", "COX5A", "COX6A1", "COX6C", "NDUFA1", "NDUFA2", "NDUFA4",
    "NDUFAB1", "NDUFB2", "NDUFB3", "NDUFS1", "NDUFS2", "NDUFV1", "UQCRC1",
    "UQCRC2", "UQCRH", "SDHA", "SDHB", "SDHC", "SDHD", "CYCS"
]

# Filter to genes present in dataset
oxphos_genes_present = [g for g in oxphos_genes if g in adata3.var_names]

# Compute OxPhos score
import scanpy as sc
sc.tl.score_genes(adata3, gene_list=oxphos_genes_present, score_name="oxphos_score")

# Check output
print(f"✅ OxPhos genes used: {len(oxphos_genes_present)}")
print(adata3.obs["oxphos_score"].describe())


✅ OxPhos genes used: 27
count    117481.000000
mean         -0.114013
std           0.775385
min          -9.954014
25%          -0.264604
50%          -0.140815
75%          -0.036540
max          54.356078
Name: oxphos_score, dtype: float64


In [21]:
## Apoptosis genes
apoptosis_genes = [
    "BAX", "BAK1", "BCL2", "BCL2L1", "BCL2L11", "CASP3", "CASP6", "CASP7",
    "CASP8", "CASP9", "TP53", "FAS", "FASLG", "TNFRSF10A", "TNFRSF10B",
    "TNFRSF1A", "TNF", "AIFM1", "APAF1", "BAD", "BID", "CFLAR", "DIABLO",
    "MCL1", "NFKB1", "NFKBIA", "TRADD", "XIAP"
]
apoptosis_genes_present = [g for g in apoptosis_genes if g in adata3.var_names]
print(f"Apoptosis genes found in dataset: {len(apoptosis_genes_present)}")
sc.tl.score_genes(adata3, gene_list=apoptosis_genes_present, score_name="apoptosis_score")
adata3.obs["apoptosis_score"]

Apoptosis genes found in dataset: 28


GGATAAGGGTCA-GSM5457199   -0.742857
CCGTGCGTACTG-GSM5457199   -1.411607
AGGTAACCTACG-GSM5457199   -1.065893
CTGTATAACCTA-GSM5457199    0.645179
AAACAGGTTTGA-GSM5457199    0.143929
                             ...   
CTCCAACCAATG-GSM5457213   -0.026250
CTCCTGAAGCAC-GSM5457213    0.004464
GTACCAGCGGCA-GSM5457213   -0.013750
TACCTCCTAAAG-GSM5457213   -0.012500
TGGTTTGTAGGG-GSM5457213    0.018214
Name: apoptosis_score, Length: 117481, dtype: float64

In [22]:
## Protooncogene score
proto_oncogenes = ['MYC', 'KRAS', 'EGFR', 'BRAF', 'AKT1', 'PIK3CA', 'CCND1', 'ERBB2', 'FGFR1', 'MDM2']

# ---------------------------------------
# 🧪 Step 2: Score Proto-oncogene activity
# Filter to genes in adata
valid_protooncogenes = [g for g in proto_oncogenes if g in adata3.var_names]
sc.tl.score_genes(adata3, gene_list=oxphos_genes_present, score_name="proto_oncogenescore")
print(f"✅ Found {len(valid_protooncogenes)} Proto-oncogenes in dataset: {valid_protooncogenes}")

✅ Found 10 Proto-oncogenes in dataset: ['MYC', 'KRAS', 'EGFR', 'BRAF', 'AKT1', 'PIK3CA', 'CCND1', 'ERBB2', 'FGFR1', 'MDM2']


In [29]:
print(adata3.obs.head(2))

                             sample        file  percent.mito cnv_reference  \
GGATAAGGGTCA-GSM5457199  GSM5457199  GSM5457199           0.0        normal   
CCGTGCGTACTG-GSM5457199  GSM5457199  GSM5457199           0.0        normal   

                         cnv_score  nCount_RNA  pct_counts_ribo   S_score  \
GGATAAGGGTCA-GSM5457199   3.085859       47478        18.918236  0.124871   
CCGTGCGTACTG-GSM5457199   2.003530       30785        18.080234  0.444978   

                         G2M_score phase  oxphos_score  apoptosis_score  \
GGATAAGGGTCA-GSM5457199  -1.576374     S     19.656475        -0.742857   
CCGTGCGTACTG-GSM5457199   0.286940     S     15.979866        -1.411607   

                         proto_oncogenescore  
GGATAAGGGTCA-GSM5457199            19.656475  
CCGTGCGTACTG-GSM5457199            15.979866  


In [30]:
adata3.obs.columns.tolist()

['sample',
 'file',
 'percent.mito',
 'cnv_reference',
 'cnv_score',
 'nCount_RNA',
 'pct_counts_ribo',
 'S_score',
 'G2M_score',
 'phase',
 'oxphos_score',
 'apoptosis_score',
 'proto_oncogenescore']

In [27]:
##dont run this code
columns_to_drop = [
    'n_genes_by_counts',
    'log1p_n_genes_by_counts',
    'total_counts',
    'log1p_total_counts',
    'pct_counts_in_top_50_genes',
    'pct_counts_in_top_100_genes',
    'pct_counts_in_top_200_genes',
    'pct_counts_in_top_500_genes',
    'total_counts_mt',
    'log1p_total_counts_mt',
    'pct_counts_mt'
]

# Drop from .obs
adata3.obs.drop(columns=columns_to_drop, inplace=True)
print(adata3.obs.columns.tolist())


KeyError: "['n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt'] not found in axis"

In [31]:
# Save the .obs DataFrame to CSV
output_path = "GSE180286_obs_features_new_16July.csv"
adata3.obs.to_csv(output_path)

print(f"✅ Saved to: {output_path}")

✅ Saved to: GSE180286_obs_features_new_16July.csv


In [21]:
# Save the .obs DataFrame to CSV
output_path = "GSE180286_obs_features_new_14July.csv"
adata3.obs.to_csv(output_path)

print(f"✅ Saved to: {output_path}")


✅ Saved to: GSE180286_obs_features_new_14July.csv


In [23]:
import pandas as pd

# Load the CSV
df_check = pd.read_csv("GSE180286_obs_features_new_14July.csv")

# Inspect percent.mito column
print(df_check["percent.mito"].describe())

# Count zeros vs non-zeros
zero_count = (df_check["percent.mito"] == 0).sum()
nonzero_count = (df_check["percent.mito"] != 0).sum()
print(f"🔍 Zero values: {zero_count}")
print(f"🔍 Non-zero values: {nonzero_count}")


count    117481.0
mean          0.0
std           0.0
min           0.0
25%           0.0
50%           0.0
75%           0.0
max           0.0
Name: percent.mito, dtype: float64
🔍 Zero values: 117481
🔍 Non-zero values: 0
